TODO: Pandas reading in CSV with [] in cell as nan DONE
TODO: np.nan solution not fixing it DONE

In [1]:
import pandas as pd
import numpy as np
import os
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from __future__ import division
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tokens = pd.read_csv('stocknews//tokens.csv', keep_default_na=False)

In [3]:
tokens.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,"[georgia, downs, two, russian, warplanes, coun...","[breaking, musharraf, impeached]","[russia, today, columns, troops, roll, south, ...","[russian, tanks, moving, towards, capital, sou...","[afghan, children, raped, impunity, official, ...","[150, russian, tanks, entered, south, ossetia,...","[breaking, georgia, invades, south, ossetia, r...","[enemy, combatent, trials, nothing, sham, sali...","[georgian, troops, retreat, osettain, capital,...","[prep, georgia, war, russia]",...,"[georgia, invades, south, ossetia, russia, get...","[al, qaeda, faces, islamist, backlash]","[condoleezza, rice, us, would, act, prevent, i...","[busy, day, european, union, approved, new, sa...","[georgia, withdraw, 000, soldiers, iraq, help,...","[pentagon, thinks, attacking, iran, bad, idea,...","[caucasus, crisis, georgia, invades, south, os...","[indian, shoe, manufactory, series, like, work]","[visitors, suffering, mental, illnesses, banne...","[help, mexico, kidnapping, surge]"
1,"[wont, america, nato, help, us, wont, help, us...","[bush, puts, foot, georgian, conflict]","[jewish, georgian, minister, thanks, israeli, ...","[georgian, army, flees, disarray, russians, ad...","[olympic, opening, ceremony, fireworks, faked]","[mossad, fraudulent, new, zealand, passports, ...","[russia, angered, israeli, military, sale, geo...","[american, citizen, living, ossetia, blames, g...","[welcome, world, war, iv, high, definition]","[georgia, move, mistake, monumental, proportions]",...,"[israel, us, behind, georgian, aggression]","[believe, tv, neither, russian, georgian, much...","[riots, still, going, montreal, canada, police...","[china, overtake, us, largest, manufacturer]","[war, south, ossetia, pics]","[israeli, physicians, group, condemns, state, ...","[russia, beaten, united, states, head, peak, oil]","[perhaps, question, georgia, russia, conflict]","[russia, much, better, war]","[come, trading, sex, food]"
2,"[remember, adorable, year, old, sang, opening,...","[russia, ends, georgia, operation]","[sexual, harassment, would, children]","[al, qa, eda, losing, support, iraq, brutal, c...","[ceasefire, georgia, putin, outmaneuvers, west]","[microsoft, intel, tried, kill, xo, 100, laptop]","[stratfor, russo, georgian, war, balance, power]","[trying, get, sense, whole, georgia, russia, w...","[us, military, surprised, timing, swiftness, r...","[beats, war, drum, iran, dumps, dollar]",...,"[troops, still, georgia, know, georgia, first,...","[russias, response, georgia, right]","[gorbachev, accuses, making, serious, blunder,...","[russia, georgia, nato, cold, war, two]","[remember, adorable, 62, year, old, led, count...","[war, georgia, israeli, connection]","[signs, point, us, encouraging, georgia, invad...","[christopher, king, argues, us, nato, behind, ...","[america, new, mexico]","[bbc, news, asia, pacific, extinction, man, cl..."
3,"[refuses, israel, weapons, attack, iran, report]","[president, ordered, attack, tskhinvali, capit...","[israel, clears, troops, killed, reuters, came...","[britain, policy, tough, drugs, pointless, say...","[body, 14, year, old, found, trunk, latest, ra...","[china, moved, 10, million, quake, survivors, ...","[bush, announces, operation, get, russia, gril...","[russian, forces, sink, georgian, ships]","[commander, navy, air, reconnaissance, squadro...","[92, cnn, readers, russia, actions, georgia, j...",...,"[elephants, extinct, 2020]","[us, humanitarian, missions, soon, georgia, ru...","[georgia, ddos, came, us, sources]","[russian, convoy, heads, georgia, violating, t...","[israeli, defence, minister, us, strike, iran]","[gorbachev, choice]","[witness, russian, forces, head, towards, tbil...","[quarter, russians, blame, conflict, poll]","[georgian, president, says, us, military, take...","[2006, nobel, laureate, aleksander, solzhenits..."
4,"[experts, admit, legalise, drugs]","[war, south, osetia, 89,

In [4]:
tokens.iloc[277,22]

'[]'

In [5]:
# Need to feature engineering for each progressive fold of the data separately.  No leakage of information.
# Split into fourths

| Fold 1  | Fold 2  | Fold 3  | Fold 4  |
|---|---|---|---|
|  Train | Test  |   |   |
|  Train | Train  | Test  |   |
|  Train | Train  | Train  | Test  |

After substantial thought, the decision was made to split the data into fourths.  With roughly 500 days of data, there is about two years of headlines and market data, which would help reduce any seasonality that might be present in the data.  By splitting the data into fourth instead of fifths, each training and test set will 'see' at least 2 full years of news and market data.

In [6]:
# Confirm size of tokens to make sure data loaded correctly
tokens.shape

(1989, 25)

In [7]:
1989/4

497.25

In [8]:
labels = pd.read_csv('stocknews//labels.csv', header=None)

http://stackoverflow.com/questions/28382735/python-pandas-does-not-read-the-first-row-of-csv-file

`pd.read_csv` was cutting off the first row of labels

In [9]:
# Confirm size of labels to make sure data loaded correctly
labels.shape

(1989, 1)

In [10]:
str_tokens = pd.read_csv('stocknews//tokens_str.csv', dtype=str, keep_default_na=False)

In [11]:
str_tokens.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,georgia downs two russian warplanes countries ...,breaking musharraf impeached,russia today columns troops roll south ossetia...,russian tanks moving towards capital south oss...,afghan children raped impunity official says s...,150 russian tanks entered south ossetia whilst...,breaking georgia invades south ossetia russia ...,enemy combatent trials nothing sham salim hama...,georgian troops retreat osettain capital presu...,prep georgia war russia,...,georgia invades south ossetia russia gets invo...,al qaeda faces islamist backlash,condoleezza rice us would act prevent israeli ...,busy day european union approved new sanctions...,georgia withdraw 000 soldiers iraq help fight ...,pentagon thinks attacking iran bad idea us new...,caucasus crisis georgia invades south ossetia,indian shoe manufactory series like work,visitors suffering mental illnesses banned oly...,help mexico kidnapping surge
1,wont america nato help us wont help us help iraq,bush puts foot georgian conflict,jewish georgian minister thanks israeli traini...,georgian army flees disarray russians advance ...,olympic opening ceremony fireworks faked,mossad fraudulent new zealand passports iraq,russia angered israeli military sale georgia,american citizen living ossetia blames georgia...,welcome world war iv high definition,georgia move mistake monumental proportions,...,israel us behind georgian aggression,believe tv neither russian georgian much victims,riots still going montreal canada police murde...,china overtake us largest manufacturer,war south ossetia pics,israeli physicians group condemns state torture,russia beaten united states head peak oil,perhaps question georgia russia conflict,russia much better war,come trading sex food
2,remember adorable year old sang opening ceremo...,russia ends georgia operation,sexual harassment would children,al qa eda losing support iraq brutal crackdown...,ceasefire georgia putin outmaneuvers west,microsoft intel tried kill xo 100 laptop,stratfor russo georgian war balance power,trying get sense whole georgia russia war vote...,us military surprised timing swiftness russian...,beats war drum iran dumps dollar,...,troops still georgia know georgia first place,russias response georgia right,gorbachev accuses making serious blunder pursu...,russia georgia nato cold war two,remember adorable 62 year old led country war ...,war georgia israeli connection,signs point us encouraging georgia invade sout...,christopher king argues us nato behind georgia...,america new mexico,bbc news asia pacific extinction man climate
3,refuses israel weapons attack iran report,president ordered attack tskhinvali capital so...,israel clears troops killed reuters cameraman,britain policy tough drugs pointless says form...,body 14 year old found trunk latest ransom pai...,china moved 10 million quake survivors prefab ...,bush announces operation get russia grill yeah...,russian forces sink georgian ships,commander navy air reconnaissance squadron pro...,92 cnn readers russia actions georgia justified,...,elephants extinct 2020,us humanitarian missions soon georgia russia h...,georgia ddos came us sources,russian convoy heads georgia violating truce,israeli defence minister us strike iran,gorbachev choice,witness russian forces head towards tbilisi br...,quarter russians blame conflict poll,georgian president says us military take contr...,2006 nobel laureate aleksander solzhenitsyn ac...
4,experts admit legalise drugs,war south osetia 89 pictures made russian soldier,swedish wrestler ara abrahamian throws away me...,russia exaggerated death toll south ossetia 44...,missile killed inside pakistan may launched cia,rushdie condemns random house refusal publish ...,poland us agree missle defense deal interestin...,russians conquer tblisi bet seriously bet,russia exaggerating south ossetian death toll ...,musharraf expected resign rathe

In [12]:
str_tokens.shape

(1989, 25)

In [13]:
str_tokens.iloc[277][22]

''

# Replace NaN with '' in prior SS

In [14]:
ph = ' '

In [15]:
def replace_num(element):
    return ' '.join([('numero' if k.isdigit() else k) for k in element.split()])

In [16]:
' '.join([('NUMERO' if k.isdigit() else k) for k in ph.split()])

''

http://stackoverflow.com/questions/6905636/python-conditional-list-joins

Instead of a digit, use `'NUMERO'` as it is resistant to stemming/lemmatizing.  It's possible that headlines with numbers might contain some information.  This will prevent preprocessing from discarding the information.

In [17]:
str_tokens.iloc[277,22]

''

In [18]:
# No longer necessary
# str_tokens.replace(np.nan,' ');

In [19]:
# No longer necessary
# str_tokens.fillna(value=' ')

In [20]:
str_tokens.applymap(replace_num)

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,georgia downs two russian warplanes countries ...,breaking musharraf impeached,russia today columns troops roll south ossetia...,russian tanks moving towards capital south oss...,afghan children raped impunity official says s...,numero russian tanks entered south ossetia whi...,breaking georgia invades south ossetia russia ...,enemy combatent trials nothing sham salim hama...,georgian troops retreat osettain capital presu...,prep georgia war russia,...,georgia invades south ossetia russia gets invo...,al qaeda faces islamist backlash,condoleezza rice us would act prevent israeli ...,busy day european union approved new sanctions...,georgia withdraw numero soldiers iraq help fig...,pentagon thinks attacking iran bad idea us new...,caucasus crisis georgia invades south ossetia,indian shoe manufactory series like work,visitors suffering mental illnesses banned oly...,help mexico kidnapping surge
1,wont america nato help us wont help us help iraq,bush puts foot georgian conflict,jewish georgian minister thanks israeli traini...,georgian army flees disarray russians advance ...,olympic opening ceremony fireworks faked,mossad fraudulent new zealand passports iraq,russia angered israeli military sale georgia,american citizen living ossetia blames georgia...,welcome world war iv high definition,georgia move mistake monumental proportions,...,israel us behind georgian aggression,believe tv neither russian georgian much victims,riots still going montreal canada police murde...,china overtake us largest manufacturer,war south ossetia pics,israeli physicians group condemns state torture,russia beaten united states head peak oil,perhaps question georgia russia conflict,russia much better war,come trading sex food
2,remember adorable year old sang opening ceremo...,russia ends georgia operation,sexual harassment would children,al qa eda losing support iraq brutal crackdown...,ceasefire georgia putin outmaneuvers west,microsoft intel tried kill xo numero laptop,stratfor russo georgian war balance power,trying get sense whole georgia russia war vote...,us military surprised timing swiftness russian...,beats war drum iran dumps dollar,...,troops still georgia know georgia first place,russias response georgia right,gorbachev accuses making serious blunder pursu...,russia georgia nato cold war two,remember adorable numero year old led country ...,war georgia israeli connection,signs point us encouraging georgia invade sout...,christopher king argues us nato behind georgia...,america new mexico,bbc news asia pacific extinction man climate
3,refuses israel weapons attack iran report,president ordered attack tskhinvali capital so...,israel clears troops killed reuters cameraman,britain policy tough drugs pointless says form...,body numero year old found trunk latest ransom...,china moved numero million quake survivors pre...,bush announces operation get russia grill yeah...,russian forces sink georgian ships,commander navy air reconnaissance squadron pro...,numero cnn readers russia actions georgia just...,...,elephants extinct numero,us humanitarian missions soon georgia russia h...,georgia ddos came us sources,russian convoy heads georgia violating truce,israeli defence minister us strike iran,gorbachev choice,witness russian forces head towards tbilisi br...,quarter russians blame conflict poll,georgian president says us military take contr...,numero nobel laureate aleksander solzhenitsyn ...
4,experts admit legalise drugs,war south osetia numero pictures made russian ...,swedish wrestler ara abrahamian throws away me...,russia exaggerated death toll south ossetia nu...,missile killed inside pakistan may launched cia,rushdie condemns random house refusal publish ...,poland us agree missle defense deal interestin...,russians conquer tblisi bet seriously bet,russia exaggerating south ossetian death toll ...,musharraf expected resig

In [21]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [22]:
sstem = SnowballStemmer('english')

In [23]:
sstem.stem('NUMERO')

'numero'

In [24]:
pstem = PorterStemmer()

In [25]:
pstem.stem('NUMERO')

'numero'

In [26]:
wnl = WordNetLemmatizer()

In [27]:
wnl.lemmatize('NUMERO')

'NUMERO'

Split labels and tokens into 4 folds
Fold 1 = [:499]
Fold 2 = [499:996]
Fold 3 = [996:1493]
Fold 4 = [1493:]

Use word2vec to measure cosine similarity between classes

In [28]:
from gensim.models.doc2vec import TaggedDocument

## Split data into train and test sets

In [29]:
train_text = str_tokens.iloc[0:1493,0]; # train features

In [30]:
test_text = str_tokens.iloc[1493:,0]; # test features

In [31]:
train_labels = labels[0:1493]; # train labels

In [32]:
test_labels = labels[1493:]; # test labels

### Convert string of words into list of words

In [33]:
str_tokens.iloc[0,0].split()

['georgia',
 'downs',
 'two',
 'russian',
 'warplanes',
 'countries',
 'move',
 'brink',
 'war']

In [34]:
train_text_list = train_text.apply(lambda x: x.split())

In [35]:
test_text_list = test_text.apply(lambda x: x.split())

In [36]:
train = train_text_list.tolist()

In [37]:
test = test_text_list.tolist()

In [38]:
train_labels = train_labels.values.tolist()

In [39]:
test_labels = test_labels.values.tolist()

In [40]:
train_data = [TaggedDocument(train[k], train_labels[k]) for k in range(len(train))]

In [41]:
from gensim.models import doc2vec

In [42]:
test_data = [TaggedDocument(test[k], test_labels[k]) for k in range(len(test))]

In [43]:
model = doc2vec.Doc2Vec(size = 100, window = 300, min_count = 1, workers = 4)

In [44]:
model.build_vocab(train_data)

In [45]:
import random

In [51]:
random.seed(1)

In [52]:
for epoch in range(12):
    model.train(train_data)
    random.shuffle(train_data)

See how words compare to others in similarity

In [53]:
model.most_similar('wall')

[('grenades', 0.8392689228057861),
 ('friendly', 0.8346019983291626),
 ('salvo', 0.8320848941802979),
 ('ball', 0.8249365091323853),
 ('767th', 0.7601796388626099),
 ('approaches', 0.7562397718429565),
 ('753bc', 0.7551333904266357),
 ('birthday', 0.7531806230545044),
 ('excavation', 0.7504466772079468),
 ('founding', 0.7479474544525146)]

In [55]:
type(model)

gensim.models.doc2vec.Doc2Vec

In [60]:
if 'numero' in model:
    print(True)
else:
    print(False)

False
